Creating a portion of Model Here

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import TextVectorization

In [2]:
df = pd.read_csv(r"C:\\Users\\as041\\OneDrive\\Desktop\\NLP\\Implementation\\Dataset\\HindiTrainDataset.csv")
df = df.drop(["Unnamed: 0", "total_classes"], axis="columns")
# df = df.drop(["Unnamed: 0", "AuthorName", "AuthorChannelId", "PublishedDate", "LikeCount"], axis="columns")
df

,Comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"This <a href="" will solve everything for long ...",0,0,0,0,0,0
1,Where can I Buy link please 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣...,0,0,0,0,0,0
2,But i fall in love with you the you deliver th...,0,0,1,0,0,0
3,Apko kiss nhi chahiye kya,1,0,1,0,0,0
4,I just wanna kiss you,0,0,1,0,0,0
...,...,...,...,...,...,...,...
2085,Aap bhi aese hi kapde pahna chalu kr do wow k...,1,0,0,0,0,0
2086,tujhse achi modern to hmari bhais h ek bhi Kap...,0,0,1,0,0,0
2087,Besharam aurat,0,1,0,0,0,1
2088,Aap bohot pyari lag rahi ho,0,0,0,0,0,0


In [3]:
print(df.isna().sum())
df = df.dropna()
df

Comment          13
toxic             0
severe_toxic      0
obscene           0
threat            0
insult            0
identity_hate     0
dtype: int64


,Comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"This <a href="" will solve everything for long ...",0,0,0,0,0,0
1,Where can I Buy link please 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣...,0,0,0,0,0,0
2,But i fall in love with you the you deliver th...,0,0,1,0,0,0
3,Apko kiss nhi chahiye kya,1,0,1,0,0,0
4,I just wanna kiss you,0,0,1,0,0,0
...,...,...,...,...,...,...,...
2085,Aap bhi aese hi kapde pahna chalu kr do wow k...,1,0,0,0,0,0
2086,tujhse achi modern to hmari bhais h ek bhi Kap...,0,0,1,0,0,0
2087,Besharam aurat,0,1,0,0,0,1
2088,Aap bohot pyari lag rahi ho,0,0,0,0,0,0


In [4]:
X = df['Comment']
y = df[df.columns[1:7]].values

In [5]:
MAX_FEATURES = 10000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,   #max length of the vocabulary
                               output_sequence_length=500,
                               output_mode='int')

In [6]:
vectorizer.adapt(X.values)

In [7]:
model = tf.keras.models.load_model("C:\\Users\\as041\\OneDrive\\Desktop\\NLP\\Implementation\\ModelCreation\\hindi_toxicity_model.h5", compile= False)

Creating the real-time toxic comment detection and deletion system

In [8]:
import os
import pickle

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from googleapiclient.discovery import build

In [9]:
credentials = None

# tokens.pickle stores the user's credentials from previously successful logins
if os.path.exists("token.pickle"):
    print("Loading Credentials From File...")
    # accessing the credentials from the pickle file
    with open("token.pickle", "rb") as token:
        credentials = pickle.load(token)


# if there is no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print("Refreshing Acess Token...")
        credentials.refresh(Request())
    else:
        print("Fetching New Tokens...")
        flow = InstalledAppFlow.from_client_secrets_file(
                            "client_secrets.json", # Run the O2auth authorisation code 
                            scopes=["https://www.googleapis.com/auth/youtube.force-ssl"] #determine the scope till what you want to access
                            )
        # start running the server to collect the authorisation credentials 
        flow.run_local_server(
                        port=8080, # port on which the code should run
                        prompt="consent",  # the message which should be prompt before the user authorizing the access
                        authorization_prompt_message = ""
                        )
        #store the credentials into a variable named credentials
        credentials = flow.credentials
        # dump the credentials back into the pickle file
        with open("token.pickle", "wb") as f:
            print("Saving Credentials for Fututre Use...")
            pickle.dump(credentials, f)

Loading Credentials From File...


<b>Acessing the comments posted on a video of the Youtube Account using Credentials</b>

In [10]:
youtube = build("youtube", "v3",  credentials=credentials)

In [11]:
# checking if the comment is toxic or not
def check_comment(comment):
    vectorised_comment = vectorizer([comment])
    results = model.predict(vectorised_comment)

    count = 0
    for idx, col in enumerate(df.columns[2:]):
        if(results[0][idx]>0.5):
            count = count+1
        else:
            pass

    return count>0    

In [12]:
# Deleting the Toxic Comments
def rejectComment(idComment):
    request = youtube.comments().setModerationStatus(
        id=idComment,
        moderationStatus="rejected"
    )
    request.execute()
    return "Rejected"

In [13]:
# accessing the comments posted on the video
request = youtube.commentThreads().list(
     part="id, replies, snippet",  videoId="8JlcXr11084"
)
response = request.execute()

# printing comments posted on the video
for snip in response['items']:
    comment_id = snip['id'] 
    comment = snip['snippet']['topLevelComment']['snippet']['textDisplay']
    if(check_comment(comment)):
        print("Comment_Id: ", comment_id, "\nComment: ", comment, "\nStatus: ",  rejectComment(comment_id))
    else:
        print("Comment_Id: ", comment_id, "\nComment: ", comment, "\nStatus: Accepted")

    if snip.get('replies', None):
        for rep in snip['replies']['comments']:
            comment_id =  rep['id']
            comment =  rep['snippet']['textDisplay']
            if(check_comment(comment)):
                print("Comment_Id: ", comment_id, "\nComment: ", comment, "\nStatus: ", rejectComment(comment_id))
            else:
                print("Comment_Id: ", comment_id, "\nComment: ", comment, "\nStatus: Accepted")


1/1 [==============================] - 2s 2s/step
Comment_Id:  UgwHGt4c3W2VPWqlLB54AaABAg 
Comment:  Yeh black screen lagane ka idea kaha se aaya. 
Status: Accepted
1/1 [==============================] - 0s 95ms/step
Comment_Id:  UgwF_nnUt-kjlX0-fRd4AaABAg 
Comment:  Tum jaise logo ko video nhi banai chahiyeh 
Status:  Rejected
1/1 [==============================] - 0s 95ms/step
Comment_Id:  UgyER8tNiWYZJ9zrIr54AaABAg 
Comment:  Poora mood kharab kr diya. Kya bekar video hain 
Status:  Rejected
1/1 [==============================] - 0s 93ms/step
Comment_Id:  UgzTatzjK5dp3yCnxSp4AaABAg 
Comment:  All the best aakash😊. We are waiting for your upcoming videos. 
Status: Accepted
